In [17]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import KFold
from core import criar_modelo, ParImageGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger

In [18]:
tf.config.run_functions_eagerly(True)

os.makedirs('../model', exist_ok=True)
os.makedirs('../results', exist_ok=True)

In [19]:
# ============================
# 🔁 Carregar todos os dados
# ============================
gen_temporario = ParImageGenerator('../image/treinamento', batch_size=1, augmentacao=False)
dados = list(zip(gen_temporario.imagens, gen_temporario.labels))  # manter como lista

In [20]:
# ============================
# 🔁 K-Fold Cross-Validation
# ============================
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
fold = 1
accuracies = []
losses = []

log_path = '../results/cv_info.txt'
with open(log_path, 'w') as log_file:

    for train_idx, val_idx in kf.split(dados):
        log_file.write(f"[Fold {fold}]\n")
        log_file.write(f"Treino: {list(train_idx)}\n")
        log_file.write(f"Validação: {list(val_idx)}\n\n")

        print(f"\n🌀 Treinando Fold {fold}...")

        train_data = [dados[i] for i in train_idx]
        val_data = [dados[i] for i in val_idx]

        train_gen = ParImageGenerator(dados=train_data, batch_size=8, augmentacao=True)
        val_gen = ParImageGenerator(dados=val_data, batch_size=8, augmentacao=False)

        modelo = criar_modelo()
        modelo.compile(optimizer=tf.keras.optimizers.Adam(),
                        loss='binary_crossentropy',
                        metrics=['accuracy'])

        checkpoint = ModelCheckpoint(
            filepath=f'../model/melhor_modelo_fold{fold}.h5',
            monitor='val_accuracy',
            save_best_only=True, # salva o melhor modelo
            verbose=1
        )

        logger = CSVLogger(f'../results/historico_treinamento_fold{fold}.csv', append=False)

        modelo.fit(
            train_gen,
            validation_data=val_gen,
            epochs=15,
            # steps_per_epoch=len(train_gen),
            steps_per_epoch=2,
            callbacks=[checkpoint, logger],
            verbose=1
        )

        loss, accuracy = modelo.evaluate(val_gen, verbose=1)
        print(f"📊 [Fold {fold}] Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

        accuracies.append(accuracy)
        losses.append(loss)
        fold += 1

In [22]:
# ============================
# 📈 Resultados finais
# ============================
media_acc = np.mean(accuracies)
media_loss = np.mean(losses)

In [23]:
print(f"\n✅ Cross-validation finalizada!")
print(f"📉 Média da Loss: {media_loss:.4f}")
print(f"✅ Média da Accuracy: {media_acc:.4f}")


✅ Cross-validation finalizada!
📉 Média da Loss: 120635.5401
✅ Média da Accuracy: 0.4286


In [24]:
# Salvar resumo no log
with open(log_path, 'a') as log_file:
    log_file.write("=== Resultado Final ===\n")
    log_file.write(f"Média da Loss: {media_loss:.4f}\n")
    log_file.write(f"Média da Accuracy: {media_acc:.4f}\n")